In [ ]:
! wget https://classes.engr.oregonstate.edu/eecs/fall2023/ai534-400/unit1/hw1/hw1-data.tgz

--2023-10-23 07:42:42--  https://classes.engr.oregonstate.edu/eecs/fall2023/ai534-400/unit1/hw1/hw1-data.tgz
Resolving classes.engr.oregonstate.edu (classes.engr.oregonstate.edu)... 128.193.40.20
Connecting to classes.engr.oregonstate.edu (classes.engr.oregonstate.edu)|128.193.40.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 54557 (53K) [application/x-gzip]
Saving to: ‘hw1-data.tgz’

hw1-data.tgz        100%[===================>]  53.28K  --.-KB/s    in 0.1s    

2023-10-23 07:42:43 (508 KB/s) - ‘hw1-data.tgz’ saved [54557/54557]



In [ ]:
! tar -xzvf hw1-data.tgz

hw1-data/random_output.py
hw1-data/validate.py
hw1-data/toy.txt
hw1-data/income.train.txt.5k
hw1-data/income.dev.txt
hw1-data/income.test.blind


In [ ]:
import numpy as np

class KNN:
    def __init__(self, k=3, distance_metric='euclidean'):
        self.k = k
        self.distance_metric = distance_metric

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        y_pred = [self._predict(x) for x in X]
        return np.array(y_pred)

    def _predict(self, x):
        # Compute the distance based on the chosen metric
        if self.distance_metric == 'euclidean':
            distances = np.linalg.norm(self.X_train - x, axis=1)
        elif self.distance_metric == 'manhattan':
            distances = np.sum(np.abs(self.X_train - x), axis=1)
        else:
            raise ValueError("Invalid distance metric")

        # Use np.argpartition to get indices of the k smallest distances
        k_indices = np.argpartition(distances, self.k)[:self.k]

        # Extract the labels of the k nearest neighbor training samples
        k_nearest_labels = self.y_train[k_indices]

        # Return the most common class label using NumPy
        labels, counts = np.unique(k_nearest_labels, return_counts=True)
        most_common_label = labels[np.argmax(counts)]

        return most_common_label

    def kneighbors(self, X, return_distance=True):
        neighbors = []

        for x in X:
            # Compute the distance based on the chosen metric
            if self.distance_metric == 'euclidean':
                distances = np.linalg.norm(self.X_train - x, axis=1)
            elif self.distance_metric == 'manhattan':
                distances = np.sum(np.abs(self.X_train - x), axis=1)
            else:
                raise ValueError("Invalid distance metric")

            # Use np.argpartition to get indices of the k smallest distances
            k_indices = np.argpartition(distances, self.k)[:self.k]

            if return_distance:
                k_distances = distances[k_indices]
                neighbors.append((k_distances, k_indices))
            else:
                neighbors.append(k_indices)

        return neighbors


In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.compose import ColumnTransformer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder

In [ ]:
columns=["age","sector","edu","marriage","occupation","race","sex","hours","country","target"]
dev_columns=["age","sector","edu","marriage","occupation","race","sex","hours","country"]
dev_data_features = pd.read_csv("hw1-data/income.test.blind", sep="," , names =dev_columns )
train_data = pd.read_csv("hw1-data/income.train.txt.5k", sep=",", names=columns)
#dev_data_features = dev_data.drop('target', axis=1)
train_data_features = train_data.drop('target', axis=1)
#dev_data_label = dev_data['target']
train_data_label = train_data['target']
num_processor=MinMaxScaler(feature_range=(0,2))
cat_processor=OneHotEncoder(sparse_output=False,handle_unknown='ignore')
preprocessor=ColumnTransformer([ ('num',num_processor,['age','hours']), ('cat',cat_processor,["sector","edu","marriage","occupation","race","sex","country"]) ])
preprocessor.fit(train_data_features)
binary_train_data_features=preprocessor.transform(train_data_features)
binary_dev_data_features= preprocessor.transform(dev_data_features)
best_dev_error_rate = float('inf')
best_k = None
for k in [41]:

  knn = KNN(k=k, distance_metric='manhattan')
  knn.fit(binary_train_data_features, train_data_label)


  train_pred = knn.predict(binary_train_data_features)
  dev_pred = knn.predict(binary_dev_data_features)

  train_positive_count = sum(train_pred == " >50K")
  train_positive_rate = train_positive_count / len(train_pred) * 100

  dev_positive_count = sum(dev_pred == " >50K")
  dev_positive_rate = dev_positive_count / len(dev_pred) * 100

  #train_err = (1 - accuracy_score(train_data_label, train_pred)) * 100
 # dev_err = (1 - accuracy_score(dev_data_label, dev_pred)) * 100


  # if dev_err < best_dev_error_rate:
  #   best_dev_error_rate = dev_err
  #   best_k = k


  print(f"k={k}  (+: {train_positive_rate:.2f}%) (+: {dev_positive_rate:.2f}%)")
#print(f"\nBest development error rate: {best_dev_error_rate:.2f}% for k={best_k}")

k=41  (+: 21.06%) (+: 20.80%)


In [ ]:
data_blind=pd.read_csv("hw1-data/income.test.blind", sep="," , names =dev_columns )
data_blind['target'] = dev_pred
output_path = "income.test.predicted"
data_blind.to_csv(output_path, header=False, index=False)


In [ ]:
cat income.test.predicted | python3 'hw1-data/random_output.py' | python3 'hw1-data/validate.py'

Your file has passed the formatting test! :)
Your positive rate is 49.6%.
ERROR: Your positive rate seems too high (should be similar to train and dev).
PLEASE DOUBLE CHECK YOUR BINARIZATION AND kNN CODE.
